[GDG Ahlen / Incremental design of LLM-powered agentic applications](https://www.youtube.com/watch?v=uIQlMSX5gx4)

Resources:
- [Google AI Studio](https://python.langchain.com/docs/integrations/chat/google_generative_ai/)
    - need to get `GOOGLE_API_KEY`
- [Finage API](https://finage.co.uk/docs/api/us-stocks#stock-market-previous-close)
    - need to get `FINAGE_API_KEY`
- [LangChain: Custom tools](https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/)


# Module 2: Call tools with Gemini using LangChain

## Setup Gemini in LangChain

Important to set Experimental model of Gemini in `ChatGoogleGenerativeAI.model` 

In [6]:
# Import env variables from `.env` file
from dotenv import load_dotenv
load_dotenv()

# Check for Google GEMINI API KEY
import os
assert "GOOGLE_API_KEY" in os.environ

from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool

llm = ChatGoogleGenerativeAI(
    max_tokens=None,
    model="gemini-2.0-flash-exp",
    temperature=1,
    top_k=1,
    top_p=0.9
)


## Setup tools

In [7]:
import requests
from typing import Dict

@tool
def get_stock_data(symbol: str) -> Dict:
    """
    Get previous day's closing price for a stock symbol using Finage API
    
    Args:
        symbol (str): Stock symbol (e.g. 'AAPL' for Apple)
        
    Returns:
        Dict: Response data containing previous close price and other details
    """
    api_key = os.getenv("FINAGE_API_KEY")
    if not api_key:
        raise ValueError("FINAGE_API_KEY not found in environment variables")
        
    url = f"https://api.finage.co.uk/agg/stock/prev-close/{symbol}"
    
    params = {
        "apikey": api_key
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        raise Exception(f"Error fetching stock data: {str(e)}")




## Prompting model with tools → just returns tool calls


In [8]:
tools = [
    get_stock_data,
    # other tools
]

llm_with_tools = llm.bind_tools(tools)

llm_with_tools.invoke("What is the previous close price for AAPL?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_stock_data', 'arguments': '{"symbol": "AAPL"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-846524dc-bbcd-4076-ae7a-dbb118e17755-0', tool_calls=[{'name': 'get_stock_data', 'args': {'symbol': 'AAPL'}, 'id': '49388d75-a8ee-4d9a-a5dd-626236213ef1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 71, 'output_tokens': 8, 'total_tokens': 79, 'input_token_details': {'cache_read': 0}})

## Full interaction with tool

1. getting the user input
2. getting tool call response from model
3. invoking the tool
4. passing the tool result back to the model for final answer to the user

In [10]:
# Initiate first message from user
messages = [
    HumanMessage(content="What is the price for Apple?")
]

# Invoke model with first message
llm_response1 = llm_with_tools.invoke(messages)
messages.append(llm_response1)

# Invoke tool returned in response to the prompt
for tool_call in llm_response1.tool_calls:
    selected_tool = {
        "get_stock_data": get_stock_data,
        # other tools
    }[tool_call["name"].lower()]
    tool_response = selected_tool.invoke(tool_call)
    # Add tool response to messages
    messages.append(tool_response)

    print(tool_response)


# Invoke model with updated messages
llm_response2 = llm_with_tools.invoke(messages)

# Print final response
print(llm_response2)

content='{"symbol": "AAPL", "totalResults": 1, "results": [{"o": 236.95, "h": 242.09, "l": 230.2, "c": 241.84, "v": 56806879, "t": 1740718800000}]}' name='get_stock_data' tool_call_id='dbfe83ad-da42-42e4-ba9b-bd844afc71d9'
content='The closing price for Apple (AAPL) was 241.84.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-83d8e829-fcbc-4b8a-8ef6-81d30ea07b2d-0' usage_metadata={'input_tokens': 93, 'output_tokens': 19, 'total_tokens': 112, 'input_token_details': {'cache_read': 0}}
